## Search algorithms within Hyperopt

[Hyperopt](http://hyperopt.github.io/hyperopt/) provides 3 search algorithms:

- Randomized search
- Annealing
- Tree-structured Parzen Estimators


I find the documentation for Hyperopt quite unintuitive, so it helps to refer to the [original article](https://iopscience.iop.org/article/10.1088/1749-4699/8/1/014008/pdf) to understand the different parameters and classes.

### Procedure

To tune the hyper-parameters of our model we need to:

- define a model
- define the hyperparameter space
- define the objective function we want to minimize.
- Run the minimization

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split

import xgboost as xgb

In [2]:
"""
-> hp : define the hyperparameter space
-> fmin : optimization function
-> Trials : to evaluate the different searched hyperparameters
"""

from hyperopt import hp, fmin, rand, anneal, tpe, Trials

In [3]:

X,y = load_breast_cancer(return_X_y=True, as_frame = True)
y = y.map({0:1, 1:0})
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
y.value_counts() / len(y)

target
0    0.627417
1    0.372583
Name: count, dtype: float64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Define the hyperparameter sspace

In [6]:
#Parameters space
param_grid = {
    'n_estimators': hp.quniform('n_estimators', 200, 2500, 100),
    'max_depth': hp.quniform('max_depth', 1, 10, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(1)),
    'booster': hp.choice('booster', ['gbtree', 'dart']),
    'gamma': hp.loguniform('gamma', np.log(0.01), np.log(10)),
    'subsample': hp.uniform('subsample', 0.50, 0.90),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.50, 0.99),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.50, 0.99),
    'colsample_bynode': hp.uniform('colsample_bynode', 0.50, 0.99),
    'reg_lambda': hp.uniform('reg_lambda', 1, 20)
}

In [7]:
### Define the objective functions

def objective(params):
    param_dict = {
        'n_estimators' : int(params['n_estimators']),
        'max_depth' : int(params['max_depth']),
        'learning_rate' : params['learning_rate'],
        'booster' : params['booster'],
        'gamma' : params['gamma'],
        'subsample' : params['subsample'],
        'colsample_bytree' : params['colsample_bytree'],
        'colsample_bylevel' : params['colsample_bylevel'],
        'colsample_bynode' : params['colsample_bynode'],
        'reg_lambda' : params['reg_lambda'],
        'random_state' : 42,
    }
    gbm = xgb.XGBClassifier(**param_dict)
    score = cross_val_score(gbm, X_train, y_train, cv=5, scoring='roc_auc', n_jobs = 4).mean()
    return -score

#### Randomized Search
fmin : returns the best hyperparameters found during the search
rand performs randomized search

In [8]:
# fmin performs the minimization
# rand.suggest samples the parameters at random
# i.e., performs the random search

random_search = fmin(
    fn=objective,
    space=param_grid,
    max_evals=50,
    rstate=np.random.default_rng(42),
    algo=rand.suggest,  # randomized search
)

 44%|████▍     | 22/50 [2:05:23<2:39:34, 341.96s/trial, best loss: -0.9913997654234107] 


KeyboardInterrupt: 

In [ ]:
type(random_search)

In [ ]:
random_search

In [ ]:
#Create another dictionary to pass the search item as parameters to the model
def create_param_grid(search, booster):
    best_hp_dict = {
            'n_estimators': int(search['n_estimators']), # important int, as it takes integers only
            'max_depth': int(search['max_depth']), # important int, as it takes integers only
            'learning_rate': search['learning_rate'],
            'booster': booster,
            'gamma': search['gamma'],
            'subsample': search['subsample'],
            'colsample_bytree': search['colsample_bytree'],
            'colsample_bylevel': search['colsample_bylevel'],
            'colsample_bynode': search['colsample_bynode'],
            'random_state': 1000,
    }
    return best_hp_dict

In [ ]:
#After the search we can train the model with the best hyperparameters

best_params = create_param_grid(random_search, 'gbtree')
gbm_rand = xgb.XGBClassifier(**best_params)
gbm_rand.fit(X_train, y_train)
X_train_preds = gbm_rand.predict(X_train)[:,1]
X_test_preds = gbm_rand.predict(X_test)[:,1]
print()
print('Train AUC: ', roc_auc_score(y_train, X_train_preds))
print('Test AUC: ', roc_auc_score(y_test, X_test_preds))

### Annealing 
anneal : performs annealing method as search

In [ ]:
anneal_search = fmin(
    fn = objective,
    space = param_grid, 
    max_evals = 50,
    rstate = np.random.default_rng(42),
    algo = anneal.suggest,
)

anneal_search

In [ ]:
#After search we can train the model with the best parameters manually

best_params = create_param_grid(anneal_search, 'gbtree')
gbm_anneal = xgb.XGBClassifier(**best_params)
gbm_anneal.fit(X_train, y_train)
X_train_preds = gbm_anneal.predict(X_train)[:,1]
X_test_preds = gbm_anneal.predict(X_test)[:,1]
print()
print('Train AUC: ', roc_auc_score(y_train, X_train_preds))
print('Test AUC: ', roc_auc_score(y_test, X_test_preds))

### TPE
performs TPE search for hyperparameters

In [ ]:
tpe_search = fmin(
    fn = objective, 
    space = param_grid,
    max_evals = 50,
    rstate = np.random.default_rng(42),
    algo=tpe.suggest, #tpe
)

tpe_search

In [ ]:
#We can train the model with the best parameters manually
best_hp_dict = create_param_grid(tpe_search, 'gbtree')
gbm_tpe = xgb.XGBClassifier(**best_hp_dict)
gbm_tpe.fit(X_train, y_train)
X_train_preds = gbm_tpe.predict(X_train)[:,1]
X_test_preds = gbm_tpe.predict(X_test)[:,1]
print()
print('Train AUC: ', roc_auc_score(y_train, X_train_preds))
print('Test AUC: ', roc_auc_score(y_test, X_test_preds))